In [33]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as web
import datetime as dt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

In [34]:
import pandas_datareader as web
import datetime as dt

In [35]:
class Model:
    def __init__(self,stk_data):
        self.data, self.test_data = train_test_split(stk_data, test_size=.2, shuffle=False)
        self.scaler = MinMaxScaler(feature_range=(0,1))
        self.scaled_data = self.scaler.fit_transform(self.data['Close'].values.reshape(-1,1))
        prediction_days = 60
        self.x_train = []
        self.y_train = []
        for x in range(prediction_days, len(self.scaled_data)):
            self.x_train.append(self.scaled_data[x-prediction_days:x, 0])
            self.y_train.append(self.scaled_data[x, 0])
        self.x_train, self.y_train = np.array(self.x_train), np.array(self.y_train)
        print(self.x_train.shape)
        print(self.x_train.shape)
        
        self.x_train = np.reshape(self.x_train, (self.x_train.shape[0], self.x_train.shape[1], 1))
        #test data preparation
        self.actual_prices = self.test_data['Close'].values
        self.total_dataset = pd.concat((self.data['Close'], self.test_data['Close']), axis=0)
        self.model_inputs = self.total_dataset[len(self.total_dataset) - len(self.test_data) - prediction_days:].values
        self.model_inputs = self.model_inputs.reshape(-1,1)
        self.model_inputs = self.scaler.transform(self.model_inputs)
        self.x_test = []
        for x in range(prediction_days, len(self.model_inputs)):
            self.x_test.append(self.model_inputs[x-prediction_days:x, 0])
        self.x_test = np.array(self.x_test)
        self.x_test = np.reshape(self.x_test, (self.x_test.shape[0], self.x_test.shape[1], 1))
        return

    def create(self):
        self.model = Sequential()
        self.model.add(LSTM(units=50, return_sequences=True, input_shape=(self.x_train.shape[1], 1)))
        self.model.add(Dropout(0.2))
        self.model.add(LSTM(units=50, return_sequences=True))
        self.model.add(Dropout(0.2))
        self.model.add(LSTM(units=50))
        self.model.add(Dropout(0.2))
        self.model.add(Dense(units=1))
        self.model.compile(optimizer='adam', loss='mean_squared_error')
        self.model.fit(self.x_train, self.y_train, epochs=25, batch_size=32)
        return self.model
        
    def predict(self, model):
        self.model = model
        self.predicted_prices = self.model.predict(self.x_test)
        self.predicted_prices = self.scaler.inverse_transform(self.predicted_prices)

    def plotting(self):
        data = pd.DataFrame()
        data['Date'] = self.test_data['Date']
        data['actual'] = self.actual_prices
        data['prediction'] = self.predicted_prices
        trace1 = go.Scatter(x=data.Date, y=data.actual, text='actual prices')
        trace2 = go.Scatter(x=data.Date, y=data.prediction, text='predicted prices')
        figure = [trace1, trace2]
        fig = {'data':figure}
        return fig

    def results(self):
        prediction_days = 60
        self.real_data = [self.model_inputs[len(self.model_inputs)+1-prediction_days:len(self.model_inputs+1), 0]]
        self.real_data = np.array(self.real_data)
        self.real_data = np.reshape(
        self.real_data, (self.real_data.shape[0], self.real_data.shape[1], 1))
        prediction = self.model.predict(self.real_data)
        prediction = self.scaler.inverse_transform(prediction)
        r2 = round(r2_score(self.actual_prices, self.predicted_prices)*100)
        return {'r2': r2,'prediction': prediction[0][0]}


In [36]:
def save(tik,year=2012):
    start = dt.datetime(year,1,1)
    end = dt.datetime.now()
    data = web.DataReader(tik, 'yahoo', start, end)
    data.to_csv('stk_data.csv', index=True)
    nn = Model()
    seq = nn.create()
    seq.save(f'models/{tik}_nn')

In [37]:
tiks = ['TTC','CCIV','PLUG','Z74.SI','FCEL']

In [38]:
company = 'RELI'

In [39]:
start = dt.datetime(2012,1,1)
end = dt.datetime.now()

In [40]:
data = web.DataReader(company, 'yahoo', start, end)
data.to_csv('stk_data.csv',index=True)

In [41]:
stk_data = pd.read_csv('stk_data.csv')
        
stk_data

,Date,High,Low,Open,Close,Volume,Adj Close
0,2021-02-09,9.07,5.200,5.40,5.82,2050100,5.82
1,2021-02-10,7.00,6.000,7.00,6.49,1121500,6.49
2,2021-02-11,6.80,6.000,6.80,6.07,274500,6.07
3,2021-02-12,5.98,4.810,5.97,5.21,484200,5.21
4,2021-02-16,5.40,5.010,5.25,5.06,258100,5.06
...,...,...,...,...,...,...,...
81,2021-06-07,4.20,4.001,4.05,4.10,72100,4.10
82,2021-06-08,4.23,4.010,4.16,4.15,96100,4.15
83,2021-06-09,4.28,3.910,4.18,3.94,174800,3.94
84,2021-06-10,4.26,3.700,4.05,3.84,209000,3.84


In [42]:
help(Model)

Help on class Model in module __main__:

class Model(builtins.object)
 |  Model(stk_data)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, stk_data)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  create(self)
 |  
 |  plotting(self)
 |  
 |  predict(self, model)
 |  
 |  results(self)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [43]:
nn = Model(stk_data)

(8, 60)
(8, 60)


In [44]:
seq = nn.create()
seq.save(f'models/{company}_nn')

Epoch 1/25
1/1 [==============================] - 5s 5s/step - loss: 0.0114
Epoch 2/25
1/1 [==============================] - 0s 41ms/step - loss: 0.0076
Epoch 3/25
1/1 [==============================] - 0s 41ms/step - loss: 0.0063
Epoch 4/25
1/1 [==============================] - 0s 40ms/step - loss: 0.0087
Epoch 5/25
1/1 [==============================] - 0s 42ms/step - loss: 0.0076
Epoch 6/25
1/1 [==============================] - 0s 40ms/step - loss: 0.0065
Epoch 7/25
1/1 [==============================] - 0s 41ms/step - loss: 0.0051
Epoch 8/25
1/1 [==============================] - 0s 40ms/step - loss: 0.0051
Epoch 9/25
1/1 [==============================] - 0s 43ms/step - loss: 0.0060
Epoch 10/25
1/1 [==============================] - 0s 41ms/step - loss: 0.0061
Epoch 11/25
1/1 [==============================] - 0s 44ms/step - loss: 0.0060
Epoch 12/25
1/1 [==============================] - 0s 40ms/step - loss: 0.0055
Epoch 13/25
1/1 [==============================] - 0s 41ms/step

INFO:tensorflow:Assets written to: models/RELI_nn/assets


INFO:tensorflow:Assets written to: models/RELI_nn/assets


In [45]:
from tensorflow import keras
model = keras.models.load_model('models/RELI_nn')

In [46]:
nn.predict(model)

In [47]:
nn.results()

{'prediction': 3.1998098, 'r2': -587}

In [48]:
!zip -r ./models.zip ./models

updating: models/ (stored 0%)
updating: models/RELI_nn/ (stored 0%)
updating: models/RELI_nn/keras_metadata.pb (deflated 92%)
updating: models/RELI_nn/assets/ (stored 0%)
updating: models/RELI_nn/saved_model.pb (deflated 91%)
updating: models/RELI_nn/variables/ (stored 0%)
updating: models/RELI_nn/variables/variables.index (deflated 67%)
updating: models/RELI_nn/variables/variables.data-00000-of-00001 (deflated 7%)


In [49]:
stks = {
    'Facebook':'FB',
    'Apple':'AAPL',
    'Tesla':'TSLA',
    'Amazon':'AMZN',
    'Google':'GOOG',
    'Microsoft Corporation':'MSFT',
    'Tata Motors Limited':'TTM',
    'Netflix, Inc.':'NFLX',
    'NVIDIA Corporation':'NVDA',
    'Wipro Limited':'WIPRO.NS',
    'Dogecoin USD':'DOGE-USD',
    'EthereumClassic USD':'ETC-USD',
}
for company in stks.values():
    save(company)

TypeError: ignored

In [54]:
!pip freeze > requirements.txt
